This script is to download data from elastic platform by some features(car number,time range)
data will be store in json file

In [1]:
from typing import MutableSet
from elasticsearch import Elasticsearch
import json
from datetime import datetime
from collections import Counter
import openpyxl


# define unix timestamp
day_timestamp = 86400000
week_timestamp = day_timestamp * 7

# car vin number to search
carNo1 = "HMZABAAH4NF003291"  
# carNo2 = "HMZABAAHXMF018277" 
# json file to store data
filename = "../../data/all-info/" + carNo1 + "_510_519.json" 
# time range
start_time = "2022.5.10 00:00:00"
end_time = "2022.5.24 00:00:00"   
# TODO change time range
# convert time range to timestamp
start_timestamp = int(datetime.strptime(start_time, "%Y.%m.%d %H:%M:%S").timestamp() * 1000)
end_timestamp = int(datetime.strptime(end_time, "%Y.%m.%d %H:%M:%S").timestamp() * 1000) 
# define time gap
time_gap = end_timestamp - start_timestamp
cycle = int(time_gap/week_timestamp)
init_res = []

for i in range(cycle):
    start_time = (start_timestamp + i * week_timestamp)
    end_time = (start_time + week_timestamp)
    print(datetime.fromtimestamp(start_time/1000))
    print(datetime.fromtimestamp(end_time/1000))
    # define soc range
    start_soc = 80
    end_soc = 100
    # define elastic address & port and authentication
    es = Elasticsearch(
        hosts=[
            "http://10.0.80.13:9200",
            "http://10.0.80.14:9200",
            "http://10.0.80.15:9200",   
        ],
        http_auth=("elastic", "123456")
    )
    # start fetch
    print("start fetch data")
    # define search range and filter
    body = {
        "query": {
            "bool": {
                # must match car vin number and charge state
                "must": [
                {"match": {"vin": carNo1}},
                 {"match": {"canData.data.VCU_BMSChargeState" :3}}
                ],
                # filt the search time range
                "filter": [{"range": {"travelTime": {"gte": start_time, "lte": end_time,}}},
                           {"range": {"canData.data.BMSSOC": {"gte": start_soc, "lte": end_soc,}}}
                ],  
            }
        },
        # sort data via travel time and by order of asc
        "sort": [
        {
            "travelTime": {"order": "asc"}
        }
        ],
        
    }
    # index is the project name, size is defined size of fetched data 
    res = es.search(index="tsp-vehicle-log3*", body=body, size=20000, request_timeout=300000)
    if i == 0:
        init_res = res["hits"]["hits"]
    else:
        init_res.extend(res["hits"]["hits"])
    print("week search done")  
# print(res)     
with open(filename, "w") as file_obj:
        json.dump(init_res, file_obj)                                                                       
print("end fetch data")
# finish saving
print("success")


2022-05-10 00:00:00
2022-05-17 00:00:00
start fetch data


/tmp/ipykernel_691/105484191.py:39: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(
/tmp/ipykernel_691/105484191.py:73: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  res = es.search(index="tsp-vehicle-log3*", body=body, size=20000, request_timeout=300000)
/tmp/ipykernel_691/105484191.py:73: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="tsp-vehicle-log3*", body=body, size=20000, request_timeout=300000)


week search done
2022-05-17 00:00:00
2022-05-24 00:00:00
start fetch data
week search done
end fetch data
success
